<b> Sort the protoplast luminescence data from the xlsx output from the Glariostar platereader. 
Use 2 input excels at a time (one firefly, one nanoluc)

In [35]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob

In [127]:
def xlsx_2_csv(xlsx):  
    """ Function to read and convert xlsx file to csv file. Also return the data (name of the folder the xlsx is in)"""
    
    # Read in the xlsx file, second sheet
    file = pd.read_excel(xlsx, 'Table End point', index_col=None) 
    
    filename = os.path.basename(xlsx)
    removed_extension = os.path.splitext(filename)[0]
    path = Path(xlsx).parent #find parent directory to the one the xlsx fields are in
    date = Path(xlsx)
    
    file.to_csv(f'{path}/{removed_extension}.csv', encoding='utf-8', index=False)
    

In [184]:
def combine_csvs(input_luc,layout_csv,picogreen_csv,date,output_file_means,output_file_raw):
    """Function to combine two csv files containingg luminescence data, and label values using layout csv file (plate layout)"""
    #read in files
    luc = pd.read_csv(input_luc, header=0)
    picogreen = pd.read_csv(picogreen_csv, header=0)
    layout_df = pd.read_csv(layout_csv, header=0)
    #make new df with correct column names
    combined = luc[['Well', 'Content','Blank corrected based on Raw Data (No filter)']].copy()
    pico = picogreen[['Well', 'Content','Blank']].copy()
    #add well row and column columns
    combined['well_row'] = combined.Well.str[:1]
    combined['well_col'] = combined.Well.str[-2:]    
    combined.rename(columns = {'Content':'content', 'Blank corrected based on Raw Data (No filter)':'luminescence'}, inplace = True)
    pico['well_row'] = pico.Well.str[:1]
    pico['well_col'] = pico.Well.str[-2:]    
    pico.rename(columns = {'Content':'content', 'Blank':'luminescence'}, inplace = True)
    #prepend layout well col with a 0
    layout = layout_df.copy()
    layout['well_col'] = layout_df['well_col'].astype(str).str.zfill(width=2)
    
    # #mask any values less than 400 (turn into NaNs)  
    combined['luminescence'] = combined.luminescence.mask(combined.luminescence < 400)
    # #merge picogreen with combine
    #change df content data type to string
    combined.content = combined.content.astype(str)
    #combined_pico = pd.merge(combined, pico, how='outer')
    #merge DNA_only pico green with layout
    merged_pico = pd.merge(pico, layout, how='outer')
  
    #combined_named = pd.concat([combined, layout])
    
    #merge with layout


    combined_named = pd.merge(combined, layout, how='outer')
    #combined_pico = pd.concat([combined_named, pico])

    # #convert well_col column data type to string so it is excluded from the next bit
    combined_named.well_col = combined_named.well_col.astype(np.str)
    merged_pico.well_col = merged_pico.well_col.astype(np.str)
    # #add new column, nluc/fluc
    # combined_named['nluc/fluc'] = combined_named.nluc_luminescence / combined_named.fluc_luminescence
    #remove NaNs
    combined_named_no_null = combined_named[pd.notnull(combined_named['luminescence'])]
    merged_pico_no_null = merged_pico[pd.notnull(merged_pico['luminescence'])]
    #make new df with mean pico
    mean_pico = merged_pico_no_null[['name','luminescence']].groupby(['name',]).mean().reset_index()
    #rename name column
    mean_pico.name = mean_pico.name.str[:-9]
    #changed column name
    mean_pico.rename(columns = {'luminescence':'mean_luminescence_pico'}, inplace = True)
    #merge mean pico with combined_named_no_null
    merged_for_correction = pd.merge(combined_named_no_null,mean_pico, how='left', on='name')
    #divide luminescence by pico DNA luminescence to correct for volume of DNA
    combined_named_no_null_corrected = merged_for_correction.copy()
    combined_named_no_null_corrected['corrected_luminescence'] = merged_for_correction.luminescence/merged_for_correction['mean_luminescence_pico']

    #put DNA_only names on same
    # #add date to the data
    combined_named_no_null_date = combined_named_no_null_corrected.copy()
    combined_named_no_null_date['date'] = date
    #make csv of raw data
    combined_named_no_null_date.to_csv(output_file_raw, encoding='utf-8', index=False)
    #make new df with mean luminescence, mean picogreen
    mean = combined_named_no_null_corrected[['name','corrected_luminescence']].groupby(['name',]).mean().reset_index()


    # ######mean = combined_named_no_null[['name', 'nluc/fluc']].groupby('name').mean().reset_index()
    mean.rename(columns = {'luminescence':'mean_luminescence'}, inplace = True)
    #merge rows
    #name = pd.DataFrame({'name':mean['name'].iloc[::2].values, 'dna_name':mean['name'].iloc[1::2].values})
    #lum = pd.DataFrame({'mean_luminescence':mean['mean_luminescence'].iloc[::2].values, 'dna_mean_luminescence':mean['mean_luminescence'].iloc[1::2].values})
    #merge on index
    #new_mean = pd.merge(name,lum, left_index=True, right_index=True)
    #new_mean.groupby('name')['dna_name'].apply(','.join).reset_index()
    
    #divide mean_luminescence of TRAMP assay by the picogreen mean_luminescence to control for concentration of DNA
    #new_mean['mean_corrected_luminescence'] = new_mean['mean_luminescence']/new_mean['dna_mean_luminescence']

    # #add standard error
    #first remove DNA_only rows
    #combined_named_no_null_no_DNA = combined_named_no_null[~combined_named_no_null['name'].str.contains('DNA_only')]
    standard_error = combined_named_no_null_corrected[['name', 'corrected_luminescence']].groupby(['name']).sem().reset_index()
    # #####standard_error = combined_named_no_null[['name','nluc/fluc']].groupby('name').sem().reset_index()
    standard_error.rename(columns = {'corrected_luminescence':'standard_error'}, inplace=True)
    mean_samples = pd.merge(mean, standard_error, on=['name'])
    # #####mean_samples = pd.merge(mean, standard_error, on='name')
    # #add date of experiment
    mean_samples['date'] = date
    # #create output file
    mean_samples.to_csv(output_file_means, encoding='utf-8', index=False)
    

In [119]:
#find all xlsx files recursively in the 'to_be_sorted' folder
xlsx_filenames = glob.glob('../../data/luminescence/TRAMP/**/*.xlsx', recursive=True)

In [45]:
#run the xlsx_2_csv function across all xlsx file in to_be_sorted folder
list(map(xlsx_2_csv,xlsx_filenames))               
                 

XLRDError: No sheet named <'Table End point'>

use os.scandir when scanning a directory, this is the fastest way according to Matt

In [46]:
# input_fluc = '../../data/luminescence/to_be_sorted/24.11.19/nitrate_free_phytogel_fluc.csv'
# input_nluc = '../../data/luminescence/to_be_sorted/24.11.19/nitrate_free_phytogel_nluc.csv'
# layout = '../../data/luminescence/to_be_sorted/24.11.19/layout.csv'
# output = '../../data/luminescence/to_be_sorted/24.11.19/output_means.csv'
# output_raw = '../../data/luminescence/to_be_sorted/24.11.19/output_raw.csv'
# date = '24.11.19'

In [55]:
date = '22.6.21'
input_luc = f'../../data/luminescence/TRAMP/{date}/22.6.21_TRAMP_ARF18_usethis.csv'
picogreen_csv = f'../../data/luminescence/TRAMP/{date}/22.6.21_picogreen_ARF18probes_repeat3_usethis.csv'
layout = f'../../data/luminescence/TRAMP/{date}/layout.csv'
output = f'../../data/luminescence/TRAMP/{date}/output_means.csv'
output_raw = f'../../data/luminescence/TRAMP/{date}/output_raw.csv'


In [185]:
combine_csvs(input_luc,layout,picogreen_csv,date,output, output_raw)